# Automatyka Pojazdowa - laboratorium 1

Celem tego laboratorium jest zapoznanie z otwartoźródłowym środowiskiem [CARLA](https://carla.org/), służącym do symulacji scenariuszy ruchu pojazdów autonomicznych. W trakcie laboratoriów będziemy korzystać ze środowiska CARLA 0.9.15.

Środowisko to pozwala na symulację end-to-end planowania, sterowania, percepcji oraz koordynacji pojazdów w kontrolowanym środowisku, które pozwala na dowolną konfigurację warunków pogodowych, ruchu drogowego czy kształtu środowiska (jezdnia, pasy, przeszkody, otoczenie).

## Uruchomienie środowiska CARLA oraz CarlaViz

W katalogu `CARLA/` wewnątrz repozytorium należy wykonać komendę `docker compose up`, która uruchamia 2 serwisy: carla oraz carla-viz. Pierwszy z nich to symulatora CARLA wraz z GUI, które stanie się widoczne na ekranie. Drugi z nich to narzędzie pozwalające na wizualizację stanu symulacji w przeglądarce.

W razie błędu mówiącego o tym, że nazwa kontenera jest już zajęta, należy usunąć istniejące kontenery: `docker container rm -f carla carla-viz`

Po uruchomieniu symulacji, należy w przeglądarce otworzyć adres [`http://localhost:8080`](http://localhost:8080).

### Sterowanie

Obecne dystrybucje CARLA opierają się o silnik Unreal Engine, mają także zaimplementowaną domyślną nawigację. Można więc w trybie "ducha":
- poruszać się po mapie za pomocą klawiszy WASD
- klawiszami Q oraz E poruszać się w dół lub w górę
- rozglądać przesuwając myszą z wciśniętym lewym przyciskiem
- zmienić tempo sterowania (tj. 'czułość') rolką myszy (góra - większa czułość, dół - mniejsza czułość)

W narzędziu CarlaViz poruszać się można z pomocą myszy przy wciśniętym lewym klawiszu, a obracać kamerę podobnie, lecz przy wciśniętym prawym klawiszu.

## Przygotowanie środowiska deweloperskiego

Kolejnym krokiem jest przygotowanie środowiska wirtualnego (tzw. virtual environment) Python. Aby uprościć instalację, na potrzeby tego laboratorium zostanie wykorzystane narzędzie Anaconda. Należy uruchomić na komputerze komendę:

`conda env list`

która powinna spowodować wylistowanie dostępnych środowisk na kształt poniższego:

```bash
lpa8@lpa8:~/ws$ conda env list
# conda environments:
#
base                     /home/lpa8/miniconda3
aad                      /home/lpa8/miniconda3/envs/aad
ap-carla-venv            *  /home/lpa8/miniconda3/envs/carla-venv
```

W razie, gdyby środowisko carla-venv istniało (jak w przykładzie powyżej), należy pominąć poniższą komendę; w razie jego braku - należy je stworzyć: `conda create -n ap-carla-venv python=3.8.20`.

Następnie powinniśmy zobaczyć znak zachęty z informacją, iż środowisko jest aktywne:
```bash
(ap-carla-venv) lpa8@lpa8:~/ws$
```

Prace w ramach tej serii laboratoriów należy wykonywać w tym środowisku.

Środowisko można opuścić za pomocą: `conda deactivate` lub aktywować za pomocą: `conda activate ap-carla-venv`.

**Zainstalowanie zależności:** `pip install -r requirements.txt` (w katalogu `CARLA/` wewnątrz repozytorium).

## Przygotowanie kernela Jupyter w środowisku condy

Następnie należy utworzyć konfigurację kernela Jupyter, który będzie korzystał z utworzonego powyżej środowiska wirtualnego condy: `python -m ipykernel install --user --name=ap-carla-venv --display-name "Python (ap-carla-venv)"`

⚠️ **Krytyczne jest** ⚠️, aby wybrać to środowisko w Visual Studio Code. Można to zrobić poprzez aktywowanie okna wyboru klikając w prawym górnym rogu przycisk "Select Kernel", następnie "Jupyter Kernel..." i ostatecznie utworzoną przez nas poprzednio konfigurację "Python (ap-carla-venv)".

## Nawiązanie połączenia z CARLA

Aby połączyć się z symulatorem CARLA z API w Pythonie, należy stworzyć instancję `carla.Client`, wskazując adres hosta oraz port, na którym nasłuchuje serwer CARLA. Wykonanie poniższej komórki powinno zakończyć się sukcesem (wypisanie instancji klasy `World`, brak błędu), jeżeli dotychczasowe instrukcje zostały wykonane poprawnie. 

In [15]:
import carla
from time import sleep
import numpy as np

client = carla.Client('localhost', 2000)
client.set_timeout(10.0) # ustawienie czasu odpowiedzi serwera (w sek.) - jeżeli CARLA nie odpowie w tym czasie na dowolną z komend, to zostanie zwrócony wyjątek

world = client.get_world()
assert(world is not None)
print(world)

World(id=276406372434079408)


## Mapy

Symulacja w CARLA opiera się na koncepcie mapy. Mapa jest definicją m. in. otoczenia, obiektów oraz dróg. Możliwe jest zarówno listowanie dostępnych map, dodawanie nowych oraz ustawianie konkretnej mapy.

In [5]:
# Pobranie dostępnych map
available_maps = client.get_available_maps()
print(available_maps)

# Załadowanie konkretnej mapy
client.load_world('Town04_Opt')
world = client.get_world()

['/Game/Carla/Maps/Town01', '/Game/Carla/Maps/Town01_Opt', '/Game/Carla/Maps/Town02', '/Game/Carla/Maps/Town02_Opt', '/Game/Carla/Maps/Town03', '/Game/Carla/Maps/Town03_Opt', '/Game/Carla/Maps/Town04', '/Game/Carla/Maps/Town04_Opt', '/Game/Carla/Maps/Town05', '/Game/Carla/Maps/Town05_Opt', '/Game/Carla/Maps/Town10HD', '/Game/Carla/Maps/Town10HD_Opt']


## Odczytywanie i filtrowanie listy aktorów

API umożliwia odczyt aktorów przez metodę klasy `World`: `get_actors()`, której rezultat jest iterowalny, ale też pozwala na filtrowanie przez wywołanie metody `.filter('...')`. Argumentem jest wyrażenie przypominające [glob](https://en.wikipedia.org/wiki/Glob_(programming)):

In [6]:
# Pobranie listy wszystkich aktorów w symulacji
actors = world.get_actors()

# Wyświetlenie informacji o wszystkich aktorach
for vehicle in actors:
    print(f'Aktor: {vehicle.type_id}, ID: {vehicle.id}')

print()

# Filtrowanie obiektów typu traffic
for vehicle in actors.filter('traffic.*'):
    print(f'Encja traffic: {vehicle.type_id}, ID: {vehicle.id}')

Aktor: traffic.speed_limit.30, ID: 282
Aktor: spectator, ID: 225
Aktor: traffic.stop, ID: 226
Aktor: traffic.stop, ID: 227
Aktor: traffic.stop, ID: 228
Aktor: traffic.speed_limit.30, ID: 286
Aktor: traffic.stop, ID: 229
Aktor: traffic.stop, ID: 230
Aktor: traffic.speed_limit.30, ID: 280
Aktor: traffic.stop, ID: 231
Aktor: traffic.stop, ID: 232
Aktor: traffic.stop, ID: 233
Aktor: traffic.yield, ID: 234
Aktor: traffic.yield, ID: 235
Aktor: traffic.yield, ID: 236
Aktor: traffic.unknown, ID: 237
Aktor: traffic.unknown, ID: 238
Aktor: traffic.yield, ID: 239
Aktor: traffic.unknown, ID: 240
Aktor: traffic.unknown, ID: 241
Aktor: traffic.speed_limit.40, ID: 242
Aktor: traffic.speed_limit.40, ID: 243
Aktor: traffic.speed_limit.90, ID: 244
Aktor: traffic.speed_limit.90, ID: 245
Aktor: traffic.speed_limit.90, ID: 246
Aktor: traffic.speed_limit.90, ID: 247
Aktor: traffic.traffic_light, ID: 327
Aktor: traffic.speed_limit.90, ID: 248
Aktor: traffic.speed_limit.90, ID: 249
Aktor: traffic.traffic_ligh

## Umieszczenie pojazdu

In [ ]:
import random

# Pobranie szablonów samochodów marki Toyota
vehicle_blueprints = world.get_blueprint_library().filter('vehicle.toyota.*')
rand_vehicle_blueprint = random.choice(vehicle_blueprints)
spawn_points = world.get_map().get_spawn_points()

rand_vehicle_blueprint.set_attribute('role_name', 'ego') # ustawienie ID ego na ten pojazd - w CarlaVIZ pojawią się wskaźniki, np. przyspieszenia

# TODO: sprawdź API carla.Map.get_spawn_points() i wypisz współrzędne 3D (X, Y, Z) dla każdego punktu spawnowania
# UWAGA: ponieważ punktów jest wiele, wybierz do 10 losowych punktów
# https://stackoverflow.com/a/509295
# https://carla.readthedocs.io/en/latest/python_api/#carla.Map.get_spawn_points
for spawn_point in spawn_points:
    # print(f'Punkt spawnowania: {...}')
    ...

# Wybór losowego punktu startowego
spawn_point = random.choice(spawn_points)

# Spawnowanie pojazdu w symulacji
added_vehicle = ... # TODO: dodaj pojazd
print(f'Utworzono pojazd: {vehicle.type_id}')

# Weryfikacja
for vehicle in world.get_actors().filter('vehicle.*'):  
    print(f'Pojazd: {vehicle.type_id}, ID: {vehicle.id}')

# UWAGA: zaobserwuj, iż dodany pojazd pojawia się na mapie; poniżej, zaimplementuj, by był usuwany

sleep(2)

# TODO: usuń pojazd z symulacji
# https://carla.readthedocs.io/en/latest/python_api/#carla.World.get_actor
# https://carla.readthedocs.io/en/latest/python_api/#carla.Actor.id
...

Punkt spawnowania: -515.2499389648438, 240.95675659179688, 0.2819424271583557
Punkt spawnowania: -511.75994873046875, 242.54856872558594, 0.2819424271583557
Punkt spawnowania: -508.24993896484375, 240.94039916992188, 0.2819424271583557
Punkt spawnowania: -504.7599792480469, 242.53224182128906, 0.2819424271583557
Punkt spawnowania: -493.2300720214844, 177.6527557373047, 0.2819424271583557
Punkt spawnowania: -489.7401428222656, 179.2745819091797, 0.2819424271583557
Punkt spawnowania: -486.230224609375, 177.69642639160156, 0.2819424271583557
Punkt spawnowania: -482.7402648925781, 179.3182830810547, 0.2819424271583557
Punkt spawnowania: 332.51141357421875, -117.62440490722656, 0.48194244503974915
Punkt spawnowania: 348.0001525878906, -142.8535614013672, 0.2819424271583557
Punkt spawnowania: 351.4998474121094, -142.80618286132812, 0.2819424271583557
Punkt spawnowania: 330.3683776855469, -121.15939331054688, 0.48194244503974915
Punkt spawnowania: 248.58294677734375, -385.22991943359375, 0.28

Ellipsis

## Sterowanie ręczne pojazdem

In [ ]:
import time

# Usunięcie istniejących pojazdów
for vehicle in world.get_actors().filter('vehicle.*'):  
    vehicle.destroy()

# TODO: dodaj pojazd do symulacji
# https://carla.readthedocs.io/en/latest/python_api/#carla.World.spawn_actor
vehicle = ...

# Ustawienie ręcznego sterowania
vehicle.apply_control(carla.VehicleControl(throttle=1, steer=0.0))  # Przyspieszenie 100%, brak skrętu

# TODO: przez 10s jedź, potem zatrzymaj pojazd
...

# https://carla.readthedocs.io/en/latest/python_api/#carla.VehicleControl.brake
... # ustaw hamowanie na 100%

## Ustawienie pozycji spektatora

W CARLA spektator to pozycja / encja, którą operuje użytkownik. Można poprzez API ustawiać i pobierać jego pozycję. Aby uzyskać obiekt pośredniczący w API z instancją spektatora w symulacji, należy wywołać `world.get_spectator()`.

W CARLA istnieją 3 kluczowe klasy, pomagające w operacjach na obiektach: `Transform` (https://carla.readthedocs.io/en/0.9.15/python_api/#carla.Transform), `Location` (https://carla.readthedocs.io/en/0.9.15/python_api/#carla.Location) oraz `Rotation` (https://carla.readthedocs.io/en/0.9.15/python_api/#carla.Rotation).

`carla.Transform` przechowuje w sobie informacje o lokalizacji (`carla.Location` jako atrybut `location`) oraz orientację (`carla.Rotation` jako atrybut `rotation`).
`carla.Location` zawiera 3 składowe: `x`, `y` oraz `z` reprezentujące współrzędne w 3-wymiarowym układzie współrzędnych.
`carla.Rotation` zawiera 3 składowe będące [kątami RPY](https://pl.wikipedia.org/wiki/K%C4%85ty_RPY) reprezentującymi obrót obiektu w przestrzeni 3-wymiarowej: `pitch`, `yaw` oraz `roll` reprezentujące współrzędne w 3-wymiarowym układzie współrzędnych.

In [ ]:
# TODO: pobierz transformację pojazdu
transform = ...

# TODO: odczytaj z carla.Transform lokalizację (carla.Location) (https://carla.readthedocs.io/en/0.9.15/python_api/#carlatransform)
location = ...

# TODO: pobierz instancję obiektu pośredniczącego przez API do obsługi spektatora
spectator = ...

# TODO: ustaw spectatora na pozycję nad autem, aby było w polu widzenia
location = ...

# TODO: obróć kamerę o 45 stopni w dół - skorzystaj z wiedzy o kątach RPY
rotation = ...
...

# TODO: przesuń kamerę w tył za auto o 10m
moveBackwardsBy = 5 # m
yawRad = np.deg2rad(rotation.yaw)
offsetX = -moveBackwardsBy * np.cos(yawRad)
offsetY = -moveBackwardsBy * np.sin(yawRad)

...
...

spectator.set_transform(...)

## Pobranie wektora trajektorii pojazdu

Klasa `carla.Transform` z kolei posiada metodę `get_forward_vector` pozwalającą uzyskać wektor wskazujący na kierunek "wprzód" zgodnie z rotacją obiektu, która zwraca [`carla.Vector3D`](https://carla.readthedocs.io/en/0.9.15/python_api/#carla.Vector3D) - klasę reprezentującą 3-wymiarowy wektor `[x, y, z]` oraz posiadającą metody pomocnicze pozwalające na wykonywanie arytmetyki wektorowej.

In [ ]:
# TODO: pobierz transformację pojazdu
transform = ...

# TODO: odczytaj z carla.Transform lokalizację (carla.Location) (https://carla.readthedocs.io/en/0.9.15/python_api/#carlatransform)
location = ...

# TODO: odczytaj z transformacji lokalizację oraz wektor 'wprzód' (można go rozumieć jako wektor trajektorii pojazdu, gdyby poruszał się wprzód)
forward_vector = ...

# wektor to uporządkowana para [start, koniec]; możemy przyjąć, że location (współrzędne pojazdu) są startem
start = location
# TODO: aby wizualizacja wektora nie pojawiła się "w środku drogi / pojazdu", przesuń je w górę o 2m
...
# natomiast potrzebny jest również koniec, który równocześnie warunkuje kierunek, zwrot i długość wektora
# TODO: ustaw koniec (end) na lokalizację przesuniętą o wektor trajektorii pojazdu
end = ...

# TODO: ustaw kolor wizualizacji na fioletowy (RGB: 127, 0, 255)
color = ...

# wyrysuj wektor jako strzałkę, korzystając z https://carla.readthedocs.io/en/0.9.15/python_api/#carla.DebugHelper.draw_arrow
debugHelper = world.debug
debugHelper.draw_arrow(
    location,
    end,
    thickness=0.1,
    arrow_size=0.3,
    color=color,
    life_time=10,  # w sekundach
)

## Interaktywne sterowanie

Zrealizuj interaktywne sterowanie w pliku [`manual-control.py`](./manual-control.py).

Uruchom skrypt korzystając z `python manual-control.py`.